In [5]:
import requests
import pandas as pd
import time
import folium

In [6]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Read the geojson file
geo_taxi_zone = gpd.read_file('../../datasets/raw/taxi_raw/NYC_Taxi_Zones.geojson')
geo_taxi_zone = geo_taxi_zone[geo_taxi_zone['borough'] == 'Manhattan']
geo_taxi_zone.head()

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
9,0.0000607235737749,24,0.0469999619287,24,Bloomingdale,Manhattan,"MULTIPOLYGON (((-73.95954 40.79872, -73.96004 ..."
12,0.0000415116236727,12,0.0366613013579,12,Battery Park,Manhattan,"MULTIPOLYGON (((-74.01566 40.70483, -74.01540 ..."
13,0.000149358592917,13,0.0502813228631,13,Battery Park City,Manhattan,"MULTIPOLYGON (((-74.01244 40.71906, -74.01282 ..."
40,0.000143093037737,41,0.052793109453,41,Central Harlem,Manhattan,"MULTIPOLYGON (((-73.94774 40.80960, -73.94506 ..."


In [7]:
#calculate centroids for each taxi zone
geo_taxi_zone['centroid'] = geo_taxi_zone.geometry.centroid

#latitude and longitude from the centroids
geo_taxi_zone['latitude'] = geo_taxi_zone.centroid.y
geo_taxi_zone['longitude'] = geo_taxi_zone.centroid.x
geo_taxi_zone.head()

C:\Users\natha\AppData\Local\Temp\ipykernel_33884\3365117267.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['centroid'] = geo_taxi_zone.geometry.centroid
C:\Users\natha\AppData\Local\Temp\ipykernel_33884\3365117267.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['latitude'] = geo_taxi_zone.centroid.y
C:\Users\natha\AppData\Local\Temp\ipykernel_33884\3365117267.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['longitude'] = geo_taxi_zone.centroid.x


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,centroid,latitude,longitude
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",POINT (-73.97697 40.72375),40.723752,-73.976968
9,0.0000607235737749,24,0.0469999619287,24,Bloomingdale,Manhattan,"MULTIPOLYGON (((-73.95954 40.79872, -73.96004 ...",POINT (-73.96548 40.80197),40.801970,-73.965480
12,0.0000415116236727,12,0.0366613013579,12,Battery Park,Manhattan,"MULTIPOLYGON (((-74.01566 40.70483, -74.01540 ...",POINT (-74.01556 40.70295),40.702946,-74.015564
13,0.000149358592917,13,0.0502813228631,13,Battery Park City,Manhattan,"MULTIPOLYGON (((-74.01244 40.71906, -74.01282 ...",POINT (-74.01608 40.71204),40.712038,-74.016079
40,0.000143093037737,41,0.052793109453,41,Central Harlem,Manhattan,"MULTIPOLYGON (((-73.94774 40.80960, -73.94506 ...",POINT (-73.95129 40.80433),40.804334,-73.951292


In [54]:
# Set your Google Places API key here
api_key = 'XXX'

def get_attractions(lat, lng, radius=500):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=tourist_attraction&key={api_key}"
    response = requests.get(url)
    return response.json()

#list to store attraction data
attraction_data = []

#loop through each taxi zone to get attractions
for index, row in geo_taxi_zone.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    response = get_attractions(lat, lng)
    attractions = response.get('results', [])[:5]  # Get top 5 attractions

    for attraction in attractions:
        attraction_data.append({
            'taxi_zone': row['location_id'],
            'zone_name': row['zone'],
            'attraction_name': attraction['name'],
            'attraction_latitude': attraction['geometry']['location']['lat'],
            'attraction_longitude': attraction['geometry']['location']['lng'],
            'attraction_vicinity': attraction.get('vicinity'),
            'attraction_rating': attraction.get('rating'),
            'user_ratings_total': attraction.get('user_ratings_total'),
        })
    
    #avoid hitting API rate limits
    time.sleep(1)


attractions_df = pd.DataFrame(attraction_data)
attractions_df.head()

,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total
0,4,Alphabet City,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,185.0
1,4,Alphabet City,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0
2,4,Alphabet City,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0
3,4,Alphabet City,Orchard Alley,40.721375,-73.978441,"350 East 4th Street, New York",4.2,17.0
4,4,Alphabet City,Museum of Reclaimed Urban Space (MoRUS),40.725766,-73.977946,"155 Loisaida Avenue, New York",4.8,89.0


In [97]:
# Set your Google Places API key
API_KEY = 'XXX'

# Define function to get attractions using Google Places API Nearby Search
def get_attractions(lat, lng, radius=5000):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=tourist_attraction&key={API_KEY}"
    response = requests.get(url)
    return response.json()

# Define function to get attraction details using Google Places API Details Search
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# Select one row (taxi zone) from the GeoDataFrame for testing
row = geo_taxi_zone.iloc[0]  # Change index as needed

# Get centroid coordinates of the taxi zone
lat = row.geometry.centroid.y
lng = row.geometry.centroid.x

# Fetch attractions using Google Places API Nearby Search
response = get_attractions(lat, lng)
attractions = response.get('results', [])[:5]  # Limit to top 5 attractions per zone

# List to store attraction data
attraction_data = []

# Iterate over each attraction to fetch details
for attraction in attractions:
    place_id = attraction['place_id']
    place_details = get_place_details(place_id)

    # Extract relevant details
    attraction_data.append({
        'taxi_zone': row['location_id'],
        'zone_name': row['zone'],
        'attraction_place_id': attraction.get('place_id')  # Add place_id
        'attraction_name': place_details.get('name', 'N/A'),
        'attraction_latitude': place_details['geometry']['location']['lat'],
        'attraction_longitude': place_details['geometry']['location']['lng'],
        'attraction_vicinity': place_details.get('vicinity', 'N/A'),
        'attraction_rating': place_details.get('rating', 'N/A'),
        'user_ratings_total': place_details.get('user_ratings_total', 'N/A'),
        'attraction_address': place_details.get('adr_address', 'N/A'),
        'attraction_phone_number': place_details.get('formatted_phone_number', 'N/A'),
        'attraction_website': place_details.get('website', 'N/A'),
        'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
        'price_level': place_details.get('price_level', 'N/A'),
        'types': place_details.get('types', 'N/A'),
        'photos': place_details.get('photos', 'N/A'),
        'reviews': place_details.get('reviews', 'N/A'),
        'plus_code': place_details.get('plus_code', 'N/A'),
        'international_phone_number': place_details.get('international_phone_number', 'N/A'),
        'url': place_details.get('url', 'N/A'),
        'utc_offset': place_details.get('utc_offset', 'N/A'),
        'permanently_closed': place_details.get('permanently_closed', 'N/A'),
        'icon': place_details.get('icon', 'N/A'),
        'business_status': place_details.get('business_status', 'N/A'),
    })

# Create DataFrame from attraction data
attractions_df = pd.DataFrame(attraction_data)

# Display the DataFrame
attractions_df.head()

,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_address,attraction_phone_number,...,types,photos,reviews,plus_code,international_phone_number,url,utc_offset,permanently_closed,icon,business_status
0,4,Alphabet City,Times Square,40.757975,-73.985543,Manhattan,4.7,213478,"<span class=""locality"">Manhattan</span>, <span...",N/A,...,"[tourist_attraction, point_of_interest, establ...","[{'height': 4000, 'html_attributions': ['<a hr...","[{'author_name': 'Dler Mhamad', 'author_url': ...","{'compound_code': 'Q257+5Q Manhattan, New York...",N/A,https://maps.google.com/?cid=6132018978369701678,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
1,4,Alphabet City,Empire State Building,40.748441,-73.985664,"20 West 34th Street, New York",4.7,105402,"<span class=""street-address"">20 W 34th St.</sp...",(212) 736-3100,...,"[tourist_attraction, museum, point_of_interest...","[{'height': 408, 'html_attributions': ['<a hre...","[{'author_name': 'Chanaka Hettige', 'author_ur...","{'compound_code': 'P2X7+9P New York, NY, USA',...",+1 212-736-3100,https://maps.google.com/?cid=15074921902713971043,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
2,4,Alphabet City,Madison Square Garden,40.750504,-73.993439,New York,4.7,30418,"<span class=""locality"">New York</span>, <span ...",(212) 465-6000,...,"[tourist_attraction, stadium, point_of_interes...","[{'height': 719, 'html_attributions': ['<a hre...","[{'author_name': 'JemD HOP', 'author_url': 'ht...","{'compound_code': 'Q224+6J New York, NY, USA',...",+1 212-465-6000,https://maps.google.com/?cid=3737724789719234788,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
3,4,Alphabet City,Rockefeller Center,40.758740,-73.978674,"45 Rockefeller Plaza, New York",4.7,175727,"<span class=""street-address"">45 Rockefeller Pl...",(212) 588-8601,...,"[tourist_attraction, point_of_interest, establ...","[{'height': 4032, 'html_attributions': ['<a hr...","[{'author_name': 'Alaleh - FoodyTO', 'author_u...","{'compound_code': 'Q25C+FG New York, NY, USA',...",+1 212-588-8601,https://maps.google.com/?cid=3734087314244816034,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
4,4,Alphabet City,Radio City Music Hall,40.759976,-73.979977,"1260 6th Avenue, New York",4.8,23899,"<span class=""street-address"">1260 6th Ave</spa...",(212) 465-6000,...,"[tourist_attraction, point_of_interest, establ...","[{'height': 2000, 'html_attributions': ['<a hr...","[{'author_name': 'Adriana De Pitkin', 'author_...","{'compound_code': 'Q25C+X2 New York, NY, USA',...",+1 212-465-6000,https://maps.google.com/?cid=10575831270349789845,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL


In [9]:
# Set your Google Places API key
API_KEY = 'XXX'

# Define function to get attractions using Google Places API Nearby Search
def get_attractions(lat, lng, radius=500):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=tourist_attraction&key={API_KEY}"
    response = requests.get(url)
    return response.json()

# Define function to get attraction details using Google Places API Details Search
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# List to store all attraction data
all_attraction_data = []

# Iterate over each taxi zone in the GeoDataFrame
for index, row in geo_taxi_zone.iterrows():
    # Get centroid coordinates of the taxi zone
    lat = row.geometry.centroid.y
    lng = row.geometry.centroid.x
    
    # Fetch attractions using Google Places API Nearby Search
    response = get_attractions(lat, lng)
    attractions = response.get('results', [])[:3]  # Limit to top 5 attractions per zone

    # List to store attraction data for the current zone
    attraction_data = []

    # Iterate over each attraction to fetch details
    for attraction in attractions:
        place_id = attraction['place_id']
        place_details = get_place_details(place_id)

        # Extract relevant details
        attraction_data.append({
            'taxi_zone': row['location_id'],
            'zone_name': row['zone'],
            'attraction_place_id': attraction.get('place_id'),  # Add place_id
            'attraction_name': place_details.get('name', 'N/A'),
            'attraction_latitude': place_details['geometry']['location']['lat'],
            'attraction_longitude': place_details['geometry']['location']['lng'],
            'attraction_vicinity': place_details.get('vicinity', 'N/A'),
            'attraction_rating': place_details.get('rating', 'N/A'),
            'user_ratings_total': place_details.get('user_ratings_total', 'N/A'),
            'attraction_address': place_details.get('adr_address', 'N/A'),
            'attraction_phone_number': place_details.get('formatted_phone_number', 'N/A'),
            'attraction_website': place_details.get('website', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'price_level': place_details.get('price_level', 'N/A'),
            'types': place_details.get('types', 'N/A'),
            'photos': place_details.get('photos', 'N/A'),
            'reviews': place_details.get('reviews', 'N/A'),
            'plus_code': place_details.get('plus_code', 'N/A'),
            'international_phone_number': place_details.get('international_phone_number', 'N/A'),
            'url': place_details.get('url', 'N/A'),
            'utc_offset': place_details.get('utc_offset', 'N/A'),
            'permanently_closed': place_details.get('permanently_closed', 'N/A'),
            'icon': place_details.get('icon', 'N/A'),
            'business_status': place_details.get('business_status', 'N/A'),
        })

    # Extend all_attraction_data with attraction_data for the current zone
    all_attraction_data.extend(attraction_data)

    # Print progress (optional)
    print(f"Fetched attractions for Taxi Zone: {row['zone']}")

    # Add a sleep time to avoid hitting API rate limits
    time.sleep(1)  # Adjust sleep time as needed

# Create DataFrame from all attraction data
attractions_df = pd.DataFrame(all_attraction_data)

# Display the DataFrame
attractions_df.head()

Fetched attractions for Taxi Zone: Alphabet City
Fetched attractions for Taxi Zone: Bloomingdale
Fetched attractions for Taxi Zone: Battery Park
Fetched attractions for Taxi Zone: Battery Park City
Fetched attractions for Taxi Zone: Central Harlem
Fetched attractions for Taxi Zone: Chinatown
Fetched attractions for Taxi Zone: Central Harlem North
Fetched attractions for Taxi Zone: Central Park
Fetched attractions for Taxi Zone: Clinton East
Fetched attractions for Taxi Zone: Clinton West
Fetched attractions for Taxi Zone: East Chelsea
Fetched attractions for Taxi Zone: East Village
Fetched attractions for Taxi Zone: East Harlem North
Fetched attractions for Taxi Zone: East Harlem South
Fetched attractions for Taxi Zone: Financial District North
Fetched attractions for Taxi Zone: Financial District South
Fetched attractions for Taxi Zone: Flatiron
Fetched attractions for Taxi Zone: Hudson Sq
Fetched attractions for Taxi Zone: Garment District
Fetched attractions for Taxi Zone: Governor'

""


In [10]:
# Set your Google Places API key
API_KEY = 'XXX'

# Define function to get attractions using Google Places API Nearby Search
def get_attractions(lat, lng, radius=500):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=tourist_attraction&key={API_KEY}"
    response = requests.get(url)
    return response.json()

# Define function to get attraction details using Google Places API Details Search
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# List to store all attraction data
all_attraction_data = []

# Iterate over each taxi zone in the GeoDataFrame
for index, row in geo_taxi_zone.iterrows():
    # Get centroid coordinates of the taxi zone
    lat = row.geometry.centroid.y
    lng = row.geometry.centroid.x
    
    # Fetch attractions using Google Places API Nearby Search
    response = get_attractions(lat, lng)
    attractions = response.get('results', [])[:3]  # Limit to top 5 attractions per zone

    # List to store attraction data for the current zone
    attraction_data = []

    # Iterate over each attraction to fetch details
    for attraction in attractions:
        place_id = attraction['place_id']
        place_details = get_place_details(place_id)

        # Extract relevant details
        attraction_data.append({
            'taxi_zone': row['location_id'],
            'zone_name': row['zone'],
            'attraction_place_id': attraction.get('place_id'),  # Add place_id
            'attraction_name': place_details.get('name', 'N/A'),
            'attraction_latitude': place_details['geometry']['location']['lat'],
            'attraction_longitude': place_details['geometry']['location']['lng'],
            'attraction_vicinity': place_details.get('vicinity', 'N/A'),
            'attraction_rating': place_details.get('rating', 'N/A'),
            'user_ratings_total': place_details.get('user_ratings_total', 'N/A'),
            'attraction_address': place_details.get('adr_address', 'N/A'),
            'attraction_phone_number': place_details.get('formatted_phone_number', 'N/A'),
            'attraction_website': place_details.get('website', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'price_level': place_details.get('price_level', 'N/A'),
            'types': place_details.get('types', 'N/A'),
            'photos': place_details.get('photos', 'N/A'),
            'reviews': place_details.get('reviews', 'N/A'),
            'plus_code': place_details.get('plus_code', 'N/A'),
            'international_phone_number': place_details.get('international_phone_number', 'N/A'),
            'url': place_details.get('url', 'N/A'),
            'utc_offset': place_details.get('utc_offset', 'N/A'),
            'permanently_closed': place_details.get('permanently_closed', 'N/A'),
            'icon': place_details.get('icon', 'N/A'),
            'business_status': place_details.get('business_status', 'N/A'),
        })

    # Extend all_attraction_data with attraction_data for the current zone
    all_attraction_data.extend(attraction_data)

    # Print progress (optional)
    print(f"Fetched attractions for Taxi Zone: {row['zone']}")

    # Add a sleep time to avoid hitting API rate limits
    time.sleep(1)  # Adjust sleep time as needed

# Create DataFrame from all attraction data
attractions_df = pd.DataFrame(all_attraction_data)

# Display the DataFrame
attractions_df.head()

Fetched attractions for Taxi Zone: Alphabet City
Fetched attractions for Taxi Zone: Bloomingdale
Fetched attractions for Taxi Zone: Battery Park
Fetched attractions for Taxi Zone: Battery Park City
Fetched attractions for Taxi Zone: Central Harlem
Fetched attractions for Taxi Zone: Chinatown
Fetched attractions for Taxi Zone: Central Harlem North
Fetched attractions for Taxi Zone: Central Park
Fetched attractions for Taxi Zone: Clinton East
Fetched attractions for Taxi Zone: Clinton West
Fetched attractions for Taxi Zone: East Chelsea
Fetched attractions for Taxi Zone: East Village
Fetched attractions for Taxi Zone: East Harlem North
Fetched attractions for Taxi Zone: East Harlem South
Fetched attractions for Taxi Zone: Financial District North
Fetched attractions for Taxi Zone: Financial District South
Fetched attractions for Taxi Zone: Flatiron
Fetched attractions for Taxi Zone: Hudson Sq
Fetched attractions for Taxi Zone: Garment District
Fetched attractions for Taxi Zone: Governor'

,taxi_zone,zone_name,attraction_place_id,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_address,...,types,photos,reviews,plus_code,international_phone_number,url,utc_offset,permanently_closed,icon,business_status
0,4,Alphabet City,ChIJbxEpSHhZwokRy0E6XKUyNPk,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,186,"<span class=""street-address"">630 E 6th St</spa...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 3024, 'html_attributions': ['<a hr...","[{'author_name': 'Saveur Spot', 'author_url': ...","{'compound_code': 'P2F9+GP New York, NY, USA',...",N/A,https://maps.google.com/?cid=17957033300031324619,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
1,4,Alphabet City,ChIJ3Z6yJiJawokR_NZVoJM3X7U,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880,"<span class=""street-address"">1234 E 6th St</sp...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 7449, 'html_attributions': ['<a hr...","[{'author_name': 'Olivia Pasquinelli', 'author...","{'compound_code': 'P2CG+MJ New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=13069225750842169084,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
2,4,Alphabet City,ChIJ5ccq-HlZwokRf-uTkQ219Rw,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37,"<span class=""street-address"">370 E 8th St</spa...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 2781, 'html_attributions': ['<a hr...","[{'author_name': 'Dana Ohlmeyer', 'author_url'...","{'compound_code': 'P2FF+J2 New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=2086773072237685631,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
3,24,Bloomingdale,ChIJa0R7Vjz2wokR_wv0XhIOo8Q,The Cathedral Church of St. John the Divine,40.803836,-73.961875,"1047 Amsterdam Avenue, New York",4.5,3676,"<span class=""street-address"">1047 Amsterdam Av...",...,"[tourist_attraction, church, place_of_worship,...","[{'height': 16256, 'html_attributions': ['<a h...","[{'author_name': 'F DB', 'author_url': 'https:...","{'compound_code': 'R23Q+G6 New York, NY, USA',...",+1 212-316-7540,https://maps.google.com/?cid=14169184324702964735,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
4,24,Bloomingdale,ChIJ4zvA1yT2wokRncwNUazVWQw,Straus Park,40.801848,-73.968187,"West End Ave. &, West 107th Street, New York",4.4,220,"West End Ave. &amp;, <span class=""street-addre...",...,"[park, tourist_attraction, point_of_interest, ...","[{'height': 4000, 'html_attributions': ['<a hr...","[{'author_name': 'Jullian Phipps', 'author_url...","{'compound_code': 'R22J+PP New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=889977337453333661,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL


In [11]:
attractions_df

,taxi_zone,zone_name,attraction_place_id,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_address,...,types,photos,reviews,plus_code,international_phone_number,url,utc_offset,permanently_closed,icon,business_status
0,4,Alphabet City,ChIJbxEpSHhZwokRy0E6XKUyNPk,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,186,"<span class=""street-address"">630 E 6th St</spa...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 3024, 'html_attributions': ['<a hr...","[{'author_name': 'Saveur Spot', 'author_url': ...","{'compound_code': 'P2F9+GP New York, NY, USA',...",N/A,https://maps.google.com/?cid=17957033300031324619,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
1,4,Alphabet City,ChIJ3Z6yJiJawokR_NZVoJM3X7U,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880,"<span class=""street-address"">1234 E 6th St</sp...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 7449, 'html_attributions': ['<a hr...","[{'author_name': 'Olivia Pasquinelli', 'author...","{'compound_code': 'P2CG+MJ New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=13069225750842169084,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
2,4,Alphabet City,ChIJ5ccq-HlZwokRf-uTkQ219Rw,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37,"<span class=""street-address"">370 E 8th St</spa...",...,"[tourist_attraction, park, point_of_interest, ...","[{'height': 2781, 'html_attributions': ['<a hr...","[{'author_name': 'Dana Ohlmeyer', 'author_url'...","{'compound_code': 'P2FF+J2 New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=2086773072237685631,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
3,24,Bloomingdale,ChIJa0R7Vjz2wokR_wv0XhIOo8Q,The Cathedral Church of St. John the Divine,40.803836,-73.961875,"1047 Amsterdam Avenue, New York",4.5,3676,"<span class=""street-address"">1047 Amsterdam Av...",...,"[tourist_attraction, church, place_of_worship,...","[{'height': 16256, 'html_attributions': ['<a h...","[{'author_name': 'F DB', 'author_url': 'https:...","{'compound_code': 'R23Q+G6 New York, NY, USA',...",+1 212-316-7540,https://maps.google.com/?cid=14169184324702964735,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
4,24,Bloomingdale,ChIJ4zvA1yT2wokRncwNUazVWQw,Straus Park,40.801848,-73.968187,"West End Ave. &, West 107th Street, New York",4.4,220,"West End Ave. &amp;, <span class=""street-addre...",...,"[park, tourist_attraction, point_of_interest, ...","[{'height': 4000, 'html_attributions': ['<a hr...","[{'author_name': 'Jullian Phipps', 'author_url...","{'compound_code': 'R22J+PP New York, NY, USA',...",+1 212-639-9675,https://maps.google.com/?cid=889977337453333661,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,261,World Trade Center,ChIJbVfZzhBawokRnmCT-muojBA,Trinity Church,40.707893,-74.011790,"89 Broadway, New York",4.6,5758,"<span class=""street-address"">89 Broadway</span...",...,"[tourist_attraction, church, place_of_worship,...","[{'height': 4000, 'html_attributions': ['<a hr...","[{'author_name': 'Doug Bush', 'author_url': 'h...","{'compound_code': 'PX5Q+57 New York, NY, USA',...",+1 212-602-0800,https://maps.google.com/?cid=1192513183065268382,-240,N/A,https://maps.gstatic.com/mapfiles/place_api/ic...,OPERATIONAL
195,261,World Trade Center,ChIJAaIFAxdawokR8txc3YPbD3A,Federal Hall,40.707399,-74.010209,"26 Wall Street, New York",4.5,2336,"<span class=""street-address"">26 Wall St</span>...",...,"[museum, tourist_attraction, point_of_interest...","[{'height': 3745, 'html_attributions': ['<a hr...","[{'author_name': 'Amber Longdon', 'author_url'...","{'compound_code': 'PX4Q+XW New York, NY, USA',...",+1 212-825-6990,https://m

In [13]:
attractions_df['price_level'].unique().tolist()

['N/A', 0, 3, 4, 2]

In [16]:
attractions_df.drop('reviews', axis=1, inplace=True)

In [17]:
attractions_df.drop('plus_code', axis=1, inplace=True)

In [18]:
attractions_df.drop('permanently_closed', axis=1, inplace=True)

In [19]:
attractions_df.drop('business_status', axis=1, inplace=True)

In [20]:
attractions_df.drop('photos', axis=1, inplace=True)

In [21]:
attractions_df.drop('utc_offset', axis=1, inplace=True)

In [22]:
attractions_df.isnull().sum()

taxi_zone                     0
zone_name                     0
attraction_place_id           0
attraction_name               0
attraction_latitude           0
attraction_longitude          0
attraction_vicinity           0
attraction_rating             0
user_ratings_total            0
attraction_address            0
attraction_phone_number       0
attraction_website            0
opening_hours                 0
price_level                   0
types                         0
international_phone_number    0
url                           0
icon                          0
dtype: int64

In [23]:
# show date types
attractions_df.dtypes

taxi_zone                      object
zone_name                      object
attraction_place_id            object
attraction_name                object
attraction_latitude           float64
attraction_longitude          float64
attraction_vicinity            object
attraction_rating              object
user_ratings_total             object
attraction_address             object
attraction_phone_number        object
attraction_website             object
opening_hours                  object
price_level                    object
types                          object
international_phone_number     object
url                            object
icon                           object
dtype: object

In [24]:
attractions_df.to_csv("attractions_df_199.csv", index=False)

In [25]:
attractions_df_new = pd.read_csv('attractions_df_199.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [26]:
for col in attractions_df_new:
    print(attractions_df_new[col])

0        4
1        4
2        4
3       24
4       24
      ... 
194    261
195    261
196    262
197    262
198    262
Name: taxi_zone, Length: 199, dtype: int64
0           Alphabet City
1           Alphabet City
2           Alphabet City
3            Bloomingdale
4            Bloomingdale
              ...        
194    World Trade Center
195    World Trade Center
196        Yorkville East
197        Yorkville East
198        Yorkville East
Name: zone_name, Length: 199, dtype: object
0      ChIJbxEpSHhZwokRy0E6XKUyNPk
1      ChIJ3Z6yJiJawokR_NZVoJM3X7U
2      ChIJ5ccq-HlZwokRf-uTkQ219Rw
3      ChIJa0R7Vjz2wokR_wv0XhIOo8Q
4      ChIJ4zvA1yT2wokRncwNUazVWQw
                  ...             
194    ChIJbVfZzhBawokRnmCT-muojBA
195    ChIJAaIFAxdawokR8txc3YPbD3A
196    ChIJkz90PrdYwokRqIYPXeGN2eA
197    ChIJ42dOPLFYwokRR31RRn7rDDM
198    ChIJp74o1rZYwokRhQTeUGzqqAI
Name: attraction_place_id, Length: 199, dtype: object
0                             6BC Botanical Garden
1            Joh

In [27]:
attractions_df_new['attraction_address'][0]

'<span class="street-address">630 E 6th St</span>, <span class="locality">New York</span>, <span class="region">NY</span> <span class="postal-code">10009-6894</span>, <span class="country-name">USA</span>'

In [28]:
attractions_df_new['opening_hours'][1]

"['Monday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Tuesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Wednesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Thursday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Friday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Saturday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Sunday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM']"

In [29]:
# Example string with Unicode escape sequences
escaped_string = "['Monday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Tuesday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Wednesday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Thursday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Friday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Saturday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM', 'Sunday: 9:00\\u202fAM\\u2009–\\u200912:00\\u202fAM']"

# Remove Unicode escape sequences using replace
cleaned_string = escaped_string.replace('\\u202f', '').replace('\\u2009', '')

print(cleaned_string)

['Monday: 9:00AM–12:00AM', 'Tuesday: 9:00AM–12:00AM', 'Wednesday: 9:00AM–12:00AM', 'Thursday: 9:00AM–12:00AM', 'Friday: 9:00AM–12:00AM', 'Saturday: 9:00AM–12:00AM', 'Sunday: 9:00AM–12:00AM']


In [30]:
attractions_df_new['opening_hours'][1]

"['Monday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Tuesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Wednesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Thursday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Friday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Saturday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Sunday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM']"

In [31]:
attractions_df_new['opening_hours'].unique().tolist()

["['Monday: Closed', 'Tuesday: Closed', 'Wednesday: 4:00\\u2009–\\u20097:00\\u202fPM', 'Thursday: Closed', 'Friday: 4:00\\u2009–\\u20097:00\\u202fPM', 'Saturday: 12:00\\u2009–\\u20096:00\\u202fPM', 'Sunday: 12:00\\u2009–\\u20096:00\\u202fPM']",
 "['Monday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Tuesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Wednesday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Thursday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Friday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Saturday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM', 'Sunday: 6:00\\u202fAM\\u2009–\\u20091:00\\u202fAM']",
 "['Monday: Closed', 'Tuesday: Closed', 'Wednesday: Closed', 'Thursday: Closed', 'Friday: 8:00\\u202fAM\\u2009–\\u20095:00\\u202fPM', 'Saturday: 8:00\\u202fAM\\u2009–\\u20095:00\\u202fPM', 'Sunday: 8:00\\u202fAM\\u2009–\\u20095:00\\u202fPM']",
 "['Monday: 9:30\\u202fAM\\u2009–\\u20095:00\\u202fPM', 'Tuesday: 9:30\\u202fAM\\u2009–\\u20095:00\\u202fPM', 'Wednesday: 9:30\

In [32]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime

# Step 1: Cleaning Function
def clean_unicode_escape_sequences(escaped_string):
    if isinstance(escaped_string, float) and np.isnan(escaped_string):
        return np.nan
    return escaped_string.replace('\\u202f', ' ').replace('\\u2009', ' ')

# Apply the cleaning function to the 'opening_hours' column
attractions_df_new['cleaned_opening_hours'] = attractions_df_new['opening_hours'].apply(clean_unicode_escape_sequences)

# Step 2: Parsing Function
def parse_opening_hours(cleaned_string):
    if pd.isna(cleaned_string):
        return np.nan
    try:
        return ast.literal_eval(cleaned_string)
    except (ValueError, SyntaxError):
        return np.nan

# Apply the parsing function to the cleaned opening hours
attractions_df_new['parsed_opening_hours'] = attractions_df_new['cleaned_opening_hours'].apply(parse_opening_hours)

# Step 3: Formatting Function
def format_opening_hours(opening_hours_list):
    if isinstance(opening_hours_list, float) and np.isnan(opening_hours_list):
        # If opening hours are null, set default hours for all days
        return '0: 09:00 - 00:00, 1: 09:00 - 00:00, 2: 09:00 - 00:00, 3: 09:00 - 00:00, 4: 09:00 - 00:00, 5: 09:00 - 00:00, 6: 09:00 - 00:00'

    day_map = {
        'Monday': '0',
        'Tuesday': '1',
        'Wednesday': '2',
        'Thursday': '3',
        'Friday': '4',
        'Saturday': '5',
        'Sunday': '6'
    }

    def convert_to_24_hour(time_str):
        if time_str == 'Open 24 hours':
            return '00:00 - 23:59'
        else:
            return datetime.strptime(time_str, "%I:%M %p").strftime("%H:%M")

    formatted_hours = []
    for day_time_str in opening_hours_list:
        try:
            day, time_range = day_time_str.split(": ")
            day = day_map[day.strip()]
            if 'Open 24 hours' in time_range:
                formatted_hours.append(f"{day}: 00:00 - 23:59")
            else:
                start_time, end_time = time_range.split("–")
                start_time_24 = convert_to_24_hour(start_time.strip())
                end_time_24 = convert_to_24_hour(end_time.strip())
                formatted_hours.append(f"{day}: {start_time_24} - {end_time_24}")
        except ValueError:
            continue  # Skip any incorrectly formatted strings
    
    return ', '.join(formatted_hours)

# Apply the formatting function to the parsed opening hours
attractions_df_new['formatted_hours'] = attractions_df_new['parsed_opening_hours'].apply(format_opening_hours)

# Display the DataFrame with formatted hours
attractions_df_new[['formatted_hours']].head()

,formatted_hours
0,
1,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
2,"4: 08:00 - 17:00, 5: 08:00 - 17:00, 6: 08:00 -..."
3,"0: 09:30 - 17:00, 1: 09:30 - 17:00, 2: 09:30 -..."
4,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."


In [33]:
attractions_df_new.isnull().sum()

taxi_zone                       0
zone_name                       0
attraction_place_id             0
attraction_name                 0
attraction_latitude             0
attraction_longitude            0
attraction_vicinity             0
attraction_rating               2
user_ratings_total              2
attraction_address              0
attraction_phone_number        44
attraction_website             17
opening_hours                  53
price_level                   181
types                           0
international_phone_number     44
url                             0
icon                            0
cleaned_opening_hours          53
parsed_opening_hours           53
formatted_hours                 0
dtype: int64

In [34]:
# Display the DataFrame with formatted hours
attractions_df_new[['opening_hours', 'formatted_hours']]

,opening_hours,formatted_hours
0,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...",
1,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
2,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...","4: 08:00 - 17:00, 5: 08:00 - 17:00, 6: 08:00 -..."
3,['Monday: 9:30\u202fAM\u2009–\u20095:00\u202fP...,"0: 09:30 - 17:00, 1: 09:30 - 17:00, 2: 09:30 -..."
4,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
...,...,...
194,['Monday: 8:30\u202fAM\u2009–\u20096:00\u202fP...,"0: 08:30 - 18:00, 1: 08:30 - 18:00, 2: 08:30 -..."
195,['Monday: 10:00\u202fAM\u2009–\u20095:00\u202f...,"0: 10:00 - 17:00, 1: 10:00 - 17:00, 2: 10:00 -..."
196,['Monday: 10:30\u202fAM\u2009–\u20093:00\u202f...,"0: 10:30 - 15:00, 2: 10:30 - 15:00"
197,['Monday: 6:00\u202fAM\u2009–\u200912:00\u202f...,"0: 06:00 - 00:00, 1: 06:00 - 00:00, 2: 06:00 -..."


In [35]:
attractions_df_new.shape

(199, 21)

In [36]:
attractions_df_new

,taxi_zone,zone_name,attraction_place_id,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_address,...,attraction_website,opening_hours,price_level,types,international_phone_number,url,icon,cleaned_opening_hours,parsed_opening_hours,formatted_hours
0,4,Alphabet City,ChIJbxEpSHhZwokRy0E6XKUyNPk,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,186.0,"<span class=""street-address"">630 E 6th St</spa...",...,http://6bcgarden.org/,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...",NaN,"['tourist_attraction', 'park', 'point_of_inter...",NaN,https://maps.google.com/?cid=17957033300031324619,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...","[Monday: Closed, Tuesday: Closed, Wednesday: 4...",
1,4,Alphabet City,ChIJ3Z6yJiJawokR_NZVoJM3X7U,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0,"<span class=""street-address"">1234 E 6th St</sp...",...,http://www.nycgovparks.org/parks/east-river-pa...,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,NaN,"['tourist_attraction', 'park', 'point_of_inter...",+1 212-639-9675,https://maps.google.com/?cid=13069225750842169084,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: 6:00 AM – 1:00 AM', 'Tuesday: 6:00 A...","[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...","0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
2,4,Alphabet City,ChIJ5ccq-HlZwokRf-uTkQ219Rw,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0,"<span class=""street-address"">370 E 8th St</spa...",...,http://greenoasisnyc.org/,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...",NaN,"['tourist_attraction', 'park', 'point_of_inter...",+1 212-639-9675,https://maps.google.com/?cid=2086773072237685631,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...","[Monday: Closed, Tuesday: Closed, Wednesday: C...","4: 08:00 - 17:00, 5: 08:00 - 17:00, 6: 08:00 -..."
3,24,Bloomingdale,ChIJa0R7Vjz2wokR_wv0XhIOo8Q,The Cathedral Church of St. John the Divine,40.803836,-73.961875,"1047 Amsterdam Avenue, New York",4.5,3676.0,"<span class=""street-address"">1047 Amsterdam Av...",...,http://www.stjohndivine.org/,['Monday: 9:30\u202fAM\u2009–\u20095:00\u202fP...,NaN,"['tourist_attraction', 'church', 'place_of_wor...",+1 212-316-7540,https://maps.google.com/?cid=14169184324702964735,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: 9:30 AM – 5:00 PM', 'Tuesday: 9:30 A...","[Monday: 9:30 AM – 5:00 PM, Tuesday: 9:30 AM –...","0: 09:30 - 17:00, 1: 09:30 - 17:00, 2: 09:30 -..."
4,24,Bloomingdale,ChIJ4zvA1yT2wokRncwNUazVWQw,Straus Park,40.801848,-73.968187,"West End Ave. &, West 107th Street, New York",4.4,220.0,"West End Ave. &amp;, <span class=""street-addre...",...,https://www.nycgovparks.org/parks/straus-park/,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,NaN,"['park', 'tourist_attraction', 'point_of_inter...",+1 212-639-9675,https://maps.google.com/?cid=889977337453333661,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: 6:00 AM – 1:00 AM', 'Tuesday: 6:00 A...","[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...","0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,261,World Trade Center,ChIJbVfZzhBawokRnmCT-muojBA,Trinity Church,40.707893,-74.011790,"89 Broadway, New York",4.6,5758.0,"<span class=""street-address"">89 Broadway</span...",...,http://www.trinitywallstreet.org/,['Monday: 8:30\u202fAM\u2009–\u20096:00\u202fP...,NaN,"['tourist_attraction', 'church', 'place_of_wor...",+1 212-602-0800,https://maps.google.com/?cid=1192513183065268382,https://maps.gstatic.com/mapfiles/place_api/ic...,"['Monday: 8:30 AM – 6:00 PM', 'Tuesday: 8:30 A...","[Monday: 8:30 AM – 6:00 PM, Tuesday: 8:30 AM –...","0: 08:30 - 18:00, 1: 08:30 - 18:00, 2: 08:30 -..."
195,261,Worl

In [337]:
attractions_df_new = attractions_df_new.to_csv("attractions_df_hours.csv", index=False)

In [14]:
attractions_df_new = pd.read_csv("attractions_df_hours.csv", keep_default_na=True, delimiter=',', skipinitialspace=True)

In [37]:
# Drop duplicate attraction names
attractions_df_new = attractions_df_new.drop_duplicates(subset=['attraction_name'], keep='first')

# Drop unnecessary columns related to opening hours
columns_to_drop = ['cleaned_opening_hours', 'parsed_opening_hours','attraction_address']
attractions_df_new = attractions_df_new.drop(columns=columns_to_drop, errors='ignore')

In [38]:
import ast

# Function to remove 'establishment' and 'tourist_attraction' from types
def clean_types(types_str):
    # Convert string representation of list back to list
    types_list = ast.literal_eval(types_str)
    # Remove 'establishment' and 'tourist_attraction'
    cleaned_list = [t for t in types_list if t not in ['establishment', 'tourist_attraction']]
    return str(cleaned_list)

# Apply the function to clean the 'types' column
attractions_df_new['types'] = attractions_df_new['types'].apply(clean_types)

In [39]:
attractions_df_new['types'].unique().tolist()

["['park', 'point_of_interest']",
 "['church', 'place_of_worship', 'point_of_interest']",
 "['museum', 'point_of_interest']",
 "['point_of_interest']",
 "['courthouse', 'local_government_office', 'point_of_interest']",
 "['gym', 'health', 'point_of_interest']",
 "['art_gallery', 'point_of_interest']",
 "['movie_theater', 'museum', 'point_of_interest']",
 "['finance', 'point_of_interest']",
 "['art_gallery', 'museum', 'point_of_interest']",
 "['airport', 'travel_agency', 'point_of_interest']",
 "['art_gallery', 'museum', 'school', 'point_of_interest']",
 "['gym', 'lodging', 'health', 'point_of_interest']",
 "['museum', 'store', 'point_of_interest']",
 "['grocery_or_supermarket', 'store', 'restaurant', 'food', 'point_of_interest']",
 "['synagogue', 'place_of_worship', 'point_of_interest']",
 "['shopping_mall', 'point_of_interest']",
 "['museum', 'library', 'point_of_interest']",
 "['art_gallery', 'movie_theater', 'point_of_interest']",
 "['stadium', 'point_of_interest']",
 "['city_hall',

In [40]:
attractions_df_new.shape

(176, 18)

In [41]:
attractions_df_new.head()

,taxi_zone,zone_name,attraction_place_id,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_phone_number,attraction_website,opening_hours,price_level,types,international_phone_number,url,icon,formatted_hours
0,4,Alphabet City,ChIJbxEpSHhZwokRy0E6XKUyNPk,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,186.0,NaN,http://6bcgarden.org/,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...",NaN,"['park', 'point_of_interest']",NaN,https://maps.google.com/?cid=17957033300031324619,https://maps.gstatic.com/mapfiles/place_api/ic...,
1,4,Alphabet City,ChIJ3Z6yJiJawokR_NZVoJM3X7U,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0,(212) 639-9675,http://www.nycgovparks.org/parks/east-river-pa...,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,NaN,"['park', 'point_of_interest']",+1 212-639-9675,https://maps.google.com/?cid=13069225750842169084,https://maps.gstatic.com/mapfiles/place_api/ic...,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."
2,4,Alphabet City,ChIJ5ccq-HlZwokRf-uTkQ219Rw,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0,(212) 639-9675,http://greenoasisnyc.org/,"['Monday: Closed', 'Tuesday: Closed', 'Wednesd...",NaN,"['park', 'point_of_interest']",+1 212-639-9675,https://maps.google.com/?cid=2086773072237685631,https://maps.gstatic.com/mapfiles/place_api/ic...,"4: 08:00 - 17:00, 5: 08:00 - 17:00, 6: 08:00 -..."
3,24,Bloomingdale,ChIJa0R7Vjz2wokR_wv0XhIOo8Q,The Cathedral Church of St. John the Divine,40.803836,-73.961875,"1047 Amsterdam Avenue, New York",4.5,3676.0,(212) 316-7540,http://www.stjohndivine.org/,['Monday: 9:30\u202fAM\u2009–\u20095:00\u202fP...,NaN,"['church', 'place_of_worship', 'point_of_inter...",+1 212-316-7540,https://maps.google.com/?cid=14169184324702964735,https://maps.gstatic.com/mapfiles/place_api/ic...,"0: 09:30 - 17:00, 1: 09:30 - 17:00, 2: 09:30 -..."
4,24,Bloomingdale,ChIJ4zvA1yT2wokRncwNUazVWQw,Straus Park,40.801848,-73.968187,"West End Ave. &, West 107th Street, New York",4.4,220.0,(212) 639-9675,https://www.nycgovparks.org/parks/straus-park/,['Monday: 6:00\u202fAM\u2009–\u20091:00\u202fA...,NaN,"['park', 'point_of_interest']",+1 212-639-9675,https://maps.google.com/?cid=889977337453333661,https://maps.gstatic.com/mapfiles/place_api/ic...,"0: 06:00 - 01:00, 1: 06:00 - 01:00, 2: 06:00 -..."


In [42]:
# map centered around Manhattan
m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)

geo_taxi_zone = gpd.read_file('../../datasets/raw/taxi_raw/NYC_Taxi_Zones.geojson')
geo_taxi_zone = geo_taxi_zone[geo_taxi_zone['borough'] == 'Manhattan']

#add taxi zones polygons to the map
folium.GeoJson(geo_taxi_zone, name="Taxi Zones").add_to(m)

#iterate over each taxi zone
for index, row in geo_taxi_zone.iterrows():
    # Get the centroid of the taxi zone
    centroid = row['geometry'].centroid
    lat = centroid.y
    lng = centroid.x

#add POIs to the map
for index, row in attractions_df_new.iterrows():
    folium.Marker([row['attraction_latitude'], row['attraction_longitude']],
                  popup=row['attraction_name'],
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(m)

#save the map to an HTML file
m.save('manhattan_poi_map.html')
print("Map with taxi zones and POIs has been saved as 'manhattan_poi_map.html'")

Map with taxi zones and POIs has been saved as 'manhattan_poi_map.html'


In [48]:
# Check for duplicate place IDs
duplicate_place_ids = attractions_df_new[attractions_df_new.duplicated('attraction_place_id', keep=False)]

# Print the rows with duplicate place IDs
print("Rows with duplicate place IDs:")
duplicate_place_ids

Rows with duplicate place IDs:


,taxi_zone,zone_name,attraction_place_id,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,attraction_phone_number,attraction_website,opening_hours,price_level,types,international_phone_number,url,icon,formatted_hours


In [49]:
duplicate_place_ids.shape

(0, 18)

In [50]:
attractions_df_new.to_csv("attractions_df_199.csv", index=False)